In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/UBC-OCEAN/updated_image_ids.json
/kaggle/input/UBC-OCEAN/sample_submission.csv
/kaggle/input/UBC-OCEAN/train.csv
/kaggle/input/UBC-OCEAN/test.csv
/kaggle/input/UBC-OCEAN/test_thumbnails/41_thumbnail.png
/kaggle/input/UBC-OCEAN/train_images/14127.png
/kaggle/input/UBC-OCEAN/train_images/34649.png
/kaggle/input/UBC-OCEAN/train_images/15221.png
/kaggle/input/UBC-OCEAN/train_images/52375.png
/kaggle/input/UBC-OCEAN/train_images/17487.png
/kaggle/input/UBC-OCEAN/train_images/32112.png
/kaggle/input/UBC-OCEAN/train_images/22290.png
/kaggle/input/UBC-OCEAN/train_images/48734.png
/kaggle/input/UBC-OCEAN/train_images/39146.png
/kaggle/input/UBC-OCEAN/train_images/64950.png
/kaggle/input/UBC-OCEAN/train_images/32042.png
/kaggle/input/UBC-OCEAN/train_images/63429.png
/kaggle/input/UBC-OCEAN/train_images/13526.png
/kaggle/input/UBC-OCEAN/train_images/53859.png
/kaggle/input/UBC-OCEAN/train_images/63836.png
/kaggle/input/UBC-OCEAN/train_images/56117.png
/kaggle/input/UBC-OCEAN/train_i

In [2]:
# from huggingface_hub import notebook_login
# notebook_login()

In [3]:
import pandas as pd
from PIL import *
from transformers import ViTForImageClassification, ViTImageProcessor
from torchvision.transforms import (CenterCrop, 
                                    Compose, 
                                    Normalize, 
                                    RandomResizedCrop, 
                                    Resize, 
                                    ToTensor)
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

# Torch
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision.models import vit_b_16, ViT_B_16_Weights
from torchinfo import summary

In [4]:
train_csv = pd.read_csv("/kaggle/input/UBC-OCEAN/train.csv")

In [5]:
CLASSES = train_csv['label'].unique()
CLASSES = sorted(CLASSES)

In [6]:
id2label = {id:label for id, label in enumerate(CLASSES)}
label2id = {label:id for id,label in id2label.items()}
id2label

{0: 'CC', 1: 'EC', 2: 'HGSC', 3: 'LGSC', 4: 'MC'}

In [7]:
# os.makedirs("checkpoint")

In [8]:
# processor.save_pretrained('/kaggle/working/checkpoint')
# model.save_pretrained("/kaggle/working/checkpoint")

In [9]:
# Replace 'your_model_name' with the actual name or path of the model you want to load
model_name = "/kaggle/input/vit-model"
processor  = ViTImageProcessor.from_pretrained(model_name)
model      = ViTForImageClassification.from_pretrained(model_name)

In [10]:
test = pd.read_csv('/kaggle/input/UBC-OCEAN/test.csv')

In [11]:
normalize = Normalize(mean=processor.image_mean, std=processor.image_std)

_train_transforms = Compose([
            Resize(tuple(processor.size.values())),
            CenterCrop(tuple(processor.size.values())),
            ToTensor(),
            normalize,
])

def train_transforms(image):
    return _train_transforms(image.convert("RGB"))

In [12]:
# GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [13]:
import numpy as np
output_list = []
for id,row in test.iterrows():
    output_df = {}
    output_df['image_id'] = row['image_id']
    transformed_images=[]
    
    image = train_transforms(Image.open("/kaggle/input/UBC-OCEAN/test_images/"+str(row['image_id'])+".png")).to(device)
    transformed_images.append(image)

    batch = torch.stack(transformed_images)
    
    with torch.no_grad():
        # output = model(image.unsqueeze(0).to(device))
        outputs = model(batch.to(device))
        
    for output in outputs[0]:
        probs = torch.nn.functional.softmax(output, dim=-1)    
        conf_score = np.max(probs.cpu().detach().numpy())
        label = id2label[np.argmax(probs.cpu().detach().numpy())]     
    output_df['label'] = label   
    output_list.append(output_df)

In [14]:
output_df = pd.DataFrame(output_list)
output_df.to_csv('submission.csv', index=False)

In [15]:
output_df

,image_id,label
0,41,HGSC
